## Wczytanie bibliotek

In [14]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score
from sklearn.base import clone
from imblearn.over_sampling import SMOTE
import xgboost as xgb
import numpy as np
import mlflow

mlflow.set_tracking_uri(uri="http://127.0.0.1:8080")

## Wczytanie i przetworzenie danych
Poniższy kod nie będzie jakoś bardzo analizowany, ze względu na to, że w poprzednich notebookach ten kod był tworzony. W tym notebooku jedynie kod jest skopiowany, aby przygotowac dane do treningu różny modeli

In [9]:
df = pd.read_csv("https://pwozniak.kia.prz.edu.pl/files/uczeniemaszynowe/train_data.csv")
df.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,59,Arcanine,Fire,NaN,555,82,110,88,105,72,98,1,False
1,107,Hitmonchan,Fighting,NaN,455,47,97,84,27,121,79,1,False
2,123,Scyther,Bug,Flying,500,63,113,86,56,73,109,1,False
3,35,Clefairy,Fairy,NaN,323,67,54,46,58,62,36,1,False
4,150,MewtwoMega Mewtwo X,Psychic,Fighting,780,97,193,102,157,98,133,1,True


In [10]:
df_no_nans = df.fillna('Other')
df_no_nans.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,59,Arcanine,Fire,Other,555,82,110,88,105,72,98,1,False
1,107,Hitmonchan,Fighting,Other,455,47,97,84,27,121,79,1,False
2,123,Scyther,Bug,Flying,500,63,113,86,56,73,109,1,False
3,35,Clefairy,Fairy,Other,323,67,54,46,58,62,36,1,False
4,150,MewtwoMega Mewtwo X,Psychic,Fighting,780,97,193,102,157,98,133,1,True


In [11]:
df_clean = df_no_nans[(df_no_nans.HP > 0) & (df_no_nans.Attack > 0) & (df_no_nans.Defense > 0)].reset_index(drop=True)

columns = ['Total', 'HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed']
for column in columns:
    print(f"Wiersze w {column} które mają wartość poniżej lub równe 0: {len(df_clean[df_clean[column] <= 0])}")

Wiersze w Total które mają wartość poniżej lub równe 0: 0
Wiersze w HP które mają wartość poniżej lub równe 0: 0
Wiersze w Attack które mają wartość poniżej lub równe 0: 0
Wiersze w Defense które mają wartość poniżej lub równe 0: 0
Wiersze w Sp. Atk które mają wartość poniżej lub równe 0: 0
Wiersze w Sp. Def które mają wartość poniżej lub równe 0: 0
Wiersze w Speed które mają wartość poniżej lub równe 0: 0


In [12]:
label_enc = LabelEncoder()
df_cat = pd.DataFrame()
for column in ['Name', 'Type 1', 'Type 2']:
    df_cat[column] = label_enc.fit_transform(df_clean[column])

In [13]:
df_num = df_clean[['Total', 'HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed']]
minmax_scaler = MinMaxScaler()
columns = df_num.columns
np_num = minmax_scaler.fit_transform(df_num)
df_num_norm = pd.DataFrame(np_num, columns=columns)
df_prepared = pd.concat([df_num_norm, df_cat, df_clean['Generation']], axis=1)

## Testowanie różnych modeli
W tej części będzie testowane 7 różnych modeli:
 - Adaboosting
 - Drzewa decyzyjne
 - Gradient boosting
 - Regresja logistycznya
 - KNN
 - SVM
 - Las losowy
Modele będą testowane na wykrywaniu imion pokemonów oraz typu I. Drugi typ zostanie pominięty, ponieważ nie jest on aż tak istotny.

In [25]:
def train(clf, X, y, model_name):
    with mlflow.start_run():
        mlflow.set_tags({"Model name" : model_name,})
        for cv in [5, 10, 20]:
            print(f"Cross validation k = {cv}")
            clf_t = clone(clf)
            cv_results = cross_validate(clf_t, X, y, scoring=("accuracy", "f1_macro"), cv=cv, n_jobs=-1)
            accuracy = np.round(np.mean(cv_results["test_accuracy"]), 3)
            f1_score = np.round(np.mean(cv_results["test_f1_macro"]), 3)
            mlflow.log_metric(f"Accuracy_{cv}", accuracy)
            mlflow.log_metric(f"F1_score_{cv}", f1_score)
            
            print(f"Accuracy: {accuracy}")
            print(f"F1-score: {f1_score}")

### Podział danych
Utworzenie zmiennych niezależnych (X) oraz zmiennej zależnej (y).

In [26]:
X = df_prepared[['Total', 'HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed']]
y_pok = df_prepared.Name

### KNN

In [27]:
mlflow.set_experiment("Predict Pokemons Name")
knn = KNeighborsClassifier()
train(knn, X, y_pok, "KNN")

Cross validation k = 5
Accuracy: 0.921
F1-score: 0.919
Cross validation k = 10
Accuracy: 0.925
F1-score: 0.92
Cross validation k = 20
Accuracy: 0.923
F1-score: 0.903


C:\Users\urbii\Desktop\Projekty\Machine-learning-course\venv\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  warnings.warn(
C:\Users\urbii\Desktop\Projekty\Machine-learning-course\venv\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=20.
  warnings.warn(
2024/12/11 18:05:22 INFO mlflow.tracking._tracking_service.client: 🏃 View run glamorous-horse-329 at: http://127.0.0.1:8080/#/experiments/479905616626895318/runs/396a4414cceb4e89b90a17d72569e1e4.
2024/12/11 18:05:22 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:8080/#/experiments/479905616626895318.


Model KNN dobrze działa na tym zbiorze danych. Jego dodatkową zaletą jest duża szybkość. Walidację krzyżową dla trzech parametrów wykonał w około 266 ms.

### Logistyczna regresja

In [28]:
mlflow.set_experiment("Predict Pokemons Name")
clf_lr = LogisticRegression()
train(clf_lr, X, y_pok, "Logisitic regression")

Cross validation k = 5
Accuracy: 0.373
F1-score: 0.278
Cross validation k = 10


C:\Users\urbii\Desktop\Projekty\Machine-learning-course\venv\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  warnings.warn(


Accuracy: 0.398
F1-score: 0.297
Cross validation k = 20


C:\Users\urbii\Desktop\Projekty\Machine-learning-course\venv\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=20.
  warnings.warn(
2024/12/11 18:05:49 INFO mlflow.tracking._tracking_service.client: 🏃 View run rare-skunk-48 at: http://127.0.0.1:8080/#/experiments/479905616626895318/runs/0dc9db5cdfe748838e82695b9fdb7c4e.
2024/12/11 18:05:49 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:8080/#/experiments/479905616626895318.


Accuracy: 0.413
F1-score: 0.301


Logisyczna regresja najsłabiej radzi sobie z tymi zadaniami. Jak można zauważyć otrzymuje ostrzeżenie, że reprezentacja klasy jest mało liczebna (mamy wiele klas, ale mało przykładów na każdą klasę). Później postaram się to zmienić z pomocą augementacji danych.

### Las losowy

In [29]:
mlflow.set_experiment("Predict Pokemons Name")
clf_rf = RandomForestClassifier(random_state=42)
train(clf_rf, X, y_pok, "Random forest")

Cross validation k = 5
Accuracy: 0.978
F1-score: 0.977
Cross validation k = 10


C:\Users\urbii\Desktop\Projekty\Machine-learning-course\venv\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  warnings.warn(


Accuracy: 0.979
F1-score: 0.977
Cross validation k = 20


C:\Users\urbii\Desktop\Projekty\Machine-learning-course\venv\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=20.
  warnings.warn(
2024/12/11 18:07:22 INFO mlflow.tracking._tracking_service.client: 🏃 View run victorious-yak-430 at: http://127.0.0.1:8080/#/experiments/479905616626895318/runs/328ae44543a64d9b8971fe01d3c6cdd0.
2024/12/11 18:07:22 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:8080/#/experiments/479905616626895318.


Accuracy: 0.978
F1-score: 0.972


Jak można zauważyć las losowy uzyskuje doskonałem wyniki. Natomiast trzeba uważać, aby go nie przeuczyć. Modele oparte na architekturze drzewa mogą dość łatwo się nadmiernie dopasować.

### Drzewo decyzyjne

In [30]:
mlflow.set_experiment("Predict Pokemons Name")
clf_dt = DecisionTreeClassifier(random_state=42)
train(clf_dt, X, y_pok, "Decision tree")

Cross validation k = 5
Accuracy: 0.963
F1-score: 0.962
Cross validation k = 10


C:\Users\urbii\Desktop\Projekty\Machine-learning-course\venv\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  warnings.warn(
C:\Users\urbii\Desktop\Projekty\Machine-learning-course\venv\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=20.
  warnings.warn(


Accuracy: 0.964
F1-score: 0.961
Cross validation k = 20
Accuracy: 0.968
F1-score: 0.959


2024/12/11 18:09:48 INFO mlflow.tracking._tracking_service.client: 🏃 View run bold-ray-287 at: http://127.0.0.1:8080/#/experiments/479905616626895318/runs/553fb8a97f1e4d359597d865a1b0598f.
2024/12/11 18:09:48 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:8080/#/experiments/479905616626895318.


Drzewo decyzyjne dla zadania klasyfikacji nazw pokemonów wypada bardzo dobrze i osiąga podobne wyniki jak las losowy. Natomiast tutaj również tak jak w poprzedniej metodzie należy uważać na przeuczenie się modelu.

### SVM

In [31]:
mlflow.set_experiment("Predict Pokemons Name")
clf_svm = SVC()
train(clf_svm, X, y_pok, "SVM")

Cross validation k = 5
Accuracy: 0.927
F1-score: 0.925
Cross validation k = 10


C:\Users\urbii\Desktop\Projekty\Machine-learning-course\venv\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  warnings.warn(


Accuracy: 0.93
F1-score: 0.924
Cross validation k = 20


C:\Users\urbii\Desktop\Projekty\Machine-learning-course\venv\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=20.
  warnings.warn(
2024/12/11 18:11:13 INFO mlflow.tracking._tracking_service.client: 🏃 View run clumsy-mule-997 at: http://127.0.0.1:8080/#/experiments/479905616626895318/runs/0354ca0f8d1441209633c75dd3ab1333.
2024/12/11 18:11:13 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:8080/#/experiments/479905616626895318.


Accuracy: 0.929
F1-score: 0.91


SVM działa dośc dobrze, natomiast lekko odbiega od Lasu losowego i Drzewa decyzyjnego. Otrzymuje podobne wyniki co KNN, natomiast jest znacząco od niego wolniejszy (czas wykonywania około 2,8 s)

### Gradient boosting

In [32]:
mlflow.set_experiment("Predict Pokemons Name")
clf_gb = GradientBoostingClassifier(random_state=42)
train(clf_gb, X, y_pok, "Gradient boosting")

Cross validation k = 5
Accuracy: 0.844
F1-score: 0.842
Cross validation k = 10


C:\Users\urbii\Desktop\Projekty\Machine-learning-course\venv\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  warnings.warn(


Accuracy: 0.535
F1-score: 0.522
Cross validation k = 20


C:\Users\urbii\Desktop\Projekty\Machine-learning-course\venv\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=20.
  warnings.warn(
2024/12/11 18:19:37 INFO mlflow.tracking._tracking_service.client: 🏃 View run dashing-dove-656 at: http://127.0.0.1:8080/#/experiments/479905616626895318/runs/e026cd7aadd44ddf99e35363ce00e98b.
2024/12/11 18:19:37 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:8080/#/experiments/479905616626895318.


Accuracy: 0.506
F1-score: 0.476


Jak można zauważyć metoda gradient boosting nie najlepiej radzi sobie z danymi. Dodatkowo przez to jak ta metoda działa (tworzy drzewa, które muszą być wykonywane jeden po drugim), czas wykonania uczenia i weryfikacji jest bardzo długi (4 minuty).

### Adaboost

In [33]:
mlflow.set_experiment("Predict Pokemons Name")
clf_ab = AdaBoostClassifier(random_state=42)
train(clf_ab, X, y_pok, "Ada boosting")

Cross validation k = 5
Accuracy: 0.11
F1-score: 0.06
Cross validation k = 10


C:\Users\urbii\Desktop\Projekty\Machine-learning-course\venv\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  warnings.warn(


Accuracy: 0.106
F1-score: 0.067
Cross validation k = 20


C:\Users\urbii\Desktop\Projekty\Machine-learning-course\venv\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=20.
  warnings.warn(
2024/12/11 18:24:39 INFO mlflow.tracking._tracking_service.client: 🏃 View run loud-grub-114 at: http://127.0.0.1:8080/#/experiments/479905616626895318/runs/ff5201fc16044a9ea5aa1547834faf6b.
2024/12/11 18:24:39 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:8080/#/experiments/479905616626895318.


Accuracy: 0.041
F1-score: 0.024


Metoda adaboost kompletnie nie radzi sobie z tymi danymi. Natomiast jest zdecydowanie szybsza niż metoda Gradient boosting.

## Xgboost

In [34]:
mlflow.set_experiment("Predict Pokemons Name")
clf_xgb = xgb.XGBClassifier(random_state=42)
train(clf_xgb, X, y_pok, "Xgboost")

Cross validation k = 5
Accuracy: 0.951
F1-score: 0.949
Cross validation k = 10


C:\Users\urbii\Desktop\Projekty\Machine-learning-course\venv\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  warnings.warn(


Accuracy: 0.956
F1-score: 0.952
Cross validation k = 20


C:\Users\urbii\Desktop\Projekty\Machine-learning-course\venv\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=20.
  warnings.warn(
2024/12/11 18:27:26 INFO mlflow.tracking._tracking_service.client: 🏃 View run beautiful-yak-325 at: http://127.0.0.1:8080/#/experiments/479905616626895318/runs/e1d02f074a67470b8c2dd5b5941404c0.
2024/12/11 18:27:26 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:8080/#/experiments/479905616626895318.


Accuracy: 0.956
F1-score: 0.942


Model xgboost jest porównywalny do lasów losowych i drzew decyzyjnych. Podczas eksperymentów wynik wskazał na to, że las losowy jest najlepszym modelem. Natomiast drzewa decyzyjne były drugie. Średni wynik dla 5-krotnej walidacji krzyżowej dla lasów losowych wyniósł 97,8%, natomiast dla drzew decyzyjnych wyniósł 96,3%. Drzewa decyzyjne są 